In [ ]:
# Install all required libraries
!pip install transformers[torch] datasets rouge_score nltk evaluate sacrebleu -q

# Import everything needed
import torch
import nltk
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

# Download the 'punkt' tokenizer models from NLTK
# This is needed for sentence tokenization in the ROUGE metric
nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True) # Added punkt_tab download

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.4 MB/s eta 0:00:00


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the T5 model and tokenizer
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Load the CNN/Daily Mail dataset
dataset = load_dataset("cnn_dailymail", '3.0.0')

# --- IMPORTANT ---
# To make training faster, we'll use a small subset of the data.
# For a full, high-quality result, use the entire dataset.
train_sample = dataset["train"].shuffle(seed=42).select(range(100000))
val_sample = dataset["validation"].shuffle(seed=42).select(range(len(dataset["validation"])))
test_sample = dataset["test"].shuffle(seed=42).select(range(len(dataset["test"])))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# The T5 model was trained with this specific prefix for summarization tasks
prefix = "summarize: "

def preprocess_function(examples):
    """Tokenizes the articles (inputs) and summaries (labels)."""
    # Prepare the input text by adding the prefix
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Prepare the target summaries (labels) using the 'as_target_tokenizer' context manager
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the tokenization function to our datasets
tokenized_train = train_sample.map(preprocess_function, batched=True)
tokenized_val = val_sample.map(preprocess_function, batched=True)
tokenized_test = test_sample.map(preprocess_function, batched=True)

print("✅ Data preprocessing complete.")

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

✅ Data preprocessing complete.


In [ ]:
# Load the ROUGE and BLEU metrics from the 'evaluate' library
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    """Computes ROUGE and BLEU scores for a batch of predictions."""
    predictions, labels = eval_pred

    # Decode the generated summaries and reference summaries
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 (pad token) to decode the labels correctly
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE score expects a newline between sentences
    rouge_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    rouge_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    rouge_result = rouge_metric.compute(predictions=rouge_preds, references=rouge_labels, use_stemmer=True)
    rouge_result = {key: value * 100 for key, value in rouge_result.items()}

    # BLEU score expects a list of references for each prediction
    bleu_labels = [[label] for label in decoded_labels]
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=bleu_labels)

    # Combine all metrics into one dictionary
    combined_metrics = {**rouge_result, "bleu": bleu_result["score"]}

    return {k: round(v, 4) for k, v in combined_metrics.items()}

In [ ]:
# The DataCollator handles dynamic padding of batches
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,      # Crucial for seq2seq tasks
    fp16=torch.cuda.is_available(),  # Enable mixed-precision training if on GPU
)

# Initialize the Trainer with all components
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics, # This is where the evaluation function is passed
)

# Start training!
print("🚀 Starting the fine-tuning process...")
trainer.train()
print("✅ Fine-tuning complete!")

/tmp/ipython-input-3573289330.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


🚀 Starting the fine-tuning process...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: huzaifaashaik (huzaifaashaik-woxsen-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu
1,1.155900,1.148818,24.948900,11.777700,20.419400,23.518100,1.254800
2,1.108400,1.131835,25.172600,11.997900,20.627200,23.735200,1.299000


✅ Fine-tuning complete!


In [ ]:
print("\n--- Testing with a sample article ---")

# Select an article from the test set
article_text = test_sample[25]["article"]
reference_summary = test_sample[25]["highlights"]

# Prepare the input for the model
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer("summarize: " + article_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate the summary
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=150,
    min_length=40,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the results
print("\nOriginal Article:\n", article_text)
print("\nReference Summary:\n", reference_summary)
print("\n✨ Generated Summary:\n", generated_summary)


--- Testing with a sample article ---

Original Article:
 A Briton jailed in Burma for using an image of the Buddha to promote his bar has been ‘abandoned’ by the Foreign Office, according to human rights campaigners. Phil Blackwood was sentenced to two-and-a-half years with hard labour by a Burmese court last month after posting the mocked-up image of the Buddha wearing DJ headphones on Facebook. The tongue-in-cheek advert for a cheap-drinks night at the VGastro bar in Rangoon provoked outrage among devout Buddhists. Scroll down for video . Phil Blackwood (left) was sentenced to two-and-a-half years with hard labour by a Burmese court last month after posting a mocked-up image of Buddha wearing DJ headphones on Facebook. Human rights campaigners claim he has been 'abandoned' by the Foreign Office . The 32-year-old bar manager, who has dual New Zealand and British nationality, was found guilty of insulting religion along with the bar’s Burmese owner and another manager, despite apolog

In [ ]:
print("\n--- Starting Final Evaluation on the Test Set ---")

# The predict() method will run the model on the test set and compute metrics
results = trainer.predict(tokenized_test)

print("\n--- ROUGE & BLEU Evaluation Results ---")
print(results.metrics)


--- Starting Final Evaluation on the Test Set ---



--- ROUGE & BLEU Evaluation Results ---
{'test_loss': 1.0943365097045898, 'test_rouge1': 25.3725, 'test_rouge2': 11.9913, 'test_rougeL': 20.8247, 'test_rougeLsum': 23.8853, 'test_bleu': 1.4832, 'test_runtime': 747.4498, 'test_samples_per_second': 15.372, 'test_steps_per_second': 1.923}
